<h2 align='center' style='color:purple'>Finding best model and hyper parameter tunning using GridSearchCV</h2>

**For iris flower dataset in sklearn library, we are going to find out best model and best hyper parameters using GridSearchCV**

<img src='iris_petal_sepal.png' height=300 width=300 />

**Load iris flower dataset**

In [1]:
from sklearn import svm, datasets
iris = datasets.load_iris()

In [2]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df[47:150]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower
47,4.6,3.2,1.4,0.2,setosa
48,5.3,3.7,1.5,0.2,setosa
49,5.0,3.3,1.4,0.2,setosa
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


<h3 style='color:blue'>Approach 1: Use train_test_split and manually tune parameters by trial and error</h3>

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [6]:
model = svm.SVC(kernel='rbf',C=30,gamma='auto') # Randomly give some parameter values
model.fit(X_train,y_train)
model.score(X_test, y_test)
# Problem with this approach is the accuracy / score changes based on the samples
# That's why we will go with K-Fold CV

0.8666666666666667

<h3 style='color:blue'>Approach 2: Use K Fold Cross validation</h3>

**Manually try suppling models with different parameters to cross_val_score function with 5 fold cross validation**

In [8]:
from sklearn.model_selection import cross_val_score

In [10]:
# try different values of the parameters such as kernel, C, gamma along with K-Fold CV
# You can see 5 values from 5 iterations
# take the average of these 5 and find the average score
cross_val_score(svm.SVC(kernel='linear',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([1.        , 1.        , 0.9       , 0.96666667, 1.        ])

In [11]:
cross_val_score(svm.SVC(kernel='rbf',C=10,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.96666667, 0.96666667, 1.        ])

In [12]:
cross_val_score(svm.SVC(kernel='rbf',C=20,gamma='auto'),iris.data, iris.target, cv=5)

array([0.96666667, 1.        , 0.9       , 0.96666667, 1.        ])

**Above approach is tiresome and very manual. We can use for loop as an alternative**

In [13]:
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

{'rbf_1': 0.9800000000000001,
 'rbf_10': 0.9800000000000001,
 'rbf_20': 0.9666666666666668,
 'linear_1': 0.9800000000000001,
 'linear_10': 0.9733333333333334,
 'linear_20': 0.9666666666666666}

**From above results we can say that rbf with C=1 or 10 or linear with C=1 will give best performance**

**This is also not the best approach as if you have 4 parameters then you may have to run the loop multiple times**

**So the best approach could be to go with GridSearchCV**

<h3 style='color:blue'>Approach 3: Use GridSearchSV</h3>

**GridSearchCV does exactly same thing as for loop above but in a single line of code**

In [15]:
from sklearn.model_selection import GridSearchCV
# We are using SVM model
# It's going to do the permutations & combinations of 'C': [1,10,20] and 'kernel': ['rbf','linear']
# We are also passing the number of CVs as 5
# GridSearchCV returns return_train_score which we don't need. That's why we are setting it as False
clf = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,10,20],
    'kernel': ['rbf','linear']
}, cv=5, return_train_score=False)
clf.fit(iris.data, iris.target) # Train the model
clf.cv_results_ # Print the CV results

{'mean_fit_time': array([0.00197244, 0.00120511, 0.00129004, 0.00110931, 0.00126143,
        0.00076256]),
 'std_fit_time': array([0.00063041, 0.00042086, 0.00039582, 0.0006732 , 0.00044247,
        0.00038718]),
 'mean_score_time': array([0.0007432 , 0.00039968, 0.00066786, 0.00098495, 0.00010481,
        0.00043116]),
 'std_score_time': array([0.00022013, 0.00048951, 0.00050757, 0.00063138, 0.00020962,
        0.00053054]),
 'param_C': masked_array(data=[1, 1, 10, 10, 20, 20],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_kernel': masked_array(data=['rbf', 'linear', 'rbf', 'linear', 'rbf', 'linear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1, 'kernel': 'rbf'},
  {'C': 1, 'kernel': 'linear'},
  {'C': 10, 'kernel': 'rbf'},
  {'C': 10, 'kernel': 'linear'},
  {'C': 20, 'kernel': 'rbf'},
  {'C': 20, 'kernel': 'linear'}],


In [17]:
# A better way of viewing the CV results by using DF
# As it is 5-Fold CV you will see split0 to split4 & also mean_test_score
df = pd.DataFrame(clf.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.001972,0.000630,0.000743,0.000220,1,rbf,"{'C': 1, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
1,0.001205,0.000421,0.000400,0.000490,1,linear,"{'C': 1, 'kernel': 'linear'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
2,0.001290,0.000396,0.000668,0.000508,10,rbf,"{'C': 10, 'kernel': 'rbf'}",0.966667,1.0,0.966667,0.966667,1.0,0.980000,0.016330,1
3,0.001109,0.000673,0.000985,0.000631,10,linear,"{'C': 10, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.966667,1.0,0.973333,0.038873,4
4,0.001261,0.000442,0.000105,0.000210,20,rbf,"{'C': 20, 'kernel': 'rbf'}",0.966667,1.0,0.900000,0.966667,1.0,0.966667,0.036515,5
5,0.000763,0.000387,0.000431,0.000531,20,linear,"{'C': 20, 'kernel': 'linear'}",1.000000,1.0,0.900000,0.933333,1.0,0.966667,0.042164,6


In [20]:
# We see the important columns from above output
# first 3 rows are the best values for the parameters for best performance
df[['param_C','param_kernel','mean_test_score']]

,param_C,param_kernel,mean_test_score
0,1,rbf,0.980000
1,1,linear,0.980000
2,10,rbf,0.980000
3,10,linear,0.973333
4,20,rbf,0.966667
5,20,linear,0.966667


In [23]:
dir(clf)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_is_fitted',
 '_estimator_type',
 '_format_results',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_pairwise',
 '_required_parameters',
 '_run_search',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',
 'decision_function',
 'error_score',
 'estimator',
 'fit',
 'get_params',
 'iid',
 'inverse_transform',
 'multimetric_',
 'n_jobs',
 'n_splits_',
 'param_grid',
 'pre_dispatch',
 'predict',
 'predict_log_proba',
 'predict_proba',
 'refit',
 'refit_time_',
 'return_train_score',
 'score

In [25]:
# You can see the best parameters
clf.best_params_

{'C': 1, 'kernel': 'rbf'}

In [26]:
# You can see the best score
clf.best_score_

0.9800000000000001

** One issue with GridSearchCV is the computation cost as we may give a range for C such as 1-50 instead of just 3 values of C:[1,10,20]  **

**Use RandomizedSearchCV to reduce number of iterations and with random combination of parameters. This is useful when you have too many parameters to try and your training time is longer. It helps reduce the cost of computation**

In [31]:
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,10,20],
        'kernel': ['rbf','linear']
    }, 
    cv=5, 
    return_train_score=False, 
    n_iter=2 # We are just using 2 iterations
)
rs.fit(iris.data, iris.target)
# See the cv_results in a better view by using DFs
pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]
# it is randomly going to choose the specified parameters

,param_C,param_kernel,mean_test_score
0,20,rbf,0.966667
1,10,rbf,0.980000


**How do you choose the best model for the given problem ?**

(or)

**How about different models with different hyperparameters?**

In [34]:
# Let's try SVM, RF & Logistic Regression by using python dictionary to find out which model gives the best performance
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,10,20],
            'kernel': ['rbf','linear']
        }  
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}


In [37]:
scores = []

# This for loop is going through these dictionay values 
# And for each of the values it will use GridSearchCV which takes 1st arg as 'model' with corresponding parameters as 2nd parameter as 'params'
for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    # append the scores
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })
    
df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df

,model,best_score,best_params
0,svm,0.980000,"{'C': 1, 'kernel': 'rbf'}"
1,random_forest,0.960000,{'n_estimators': 10}
2,logistic_regression,0.966667,{'C': 5}


**Based on above, I can conclude that SVM with C=1 and kernel='rbf' is the best model for solving my problem of iris flower classification**